In [ ]:
!pip install transformers --quiet

     |████████████████████████████████| 2.2MB 18.5MB/s 
     |████████████████████████████████| 3.3MB 50.9MB/s 
     |████████████████████████████████| 870kB 51.1MB/s 


In [ ]:
import re
import json
import numpy as np
import pandas as pd

from tqdm import tqdm_notebook

import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
from transformers import Seq2SeqTrainingArguments, Seq2SeqTrainer
from transformers import EncoderDecoderModel, AutoTokenizer, pipeline
from torch.utils.data import Dataset, DataLoader, RandomSampler, SequentialSampler

from sklearn.model_selection import train_test_split

print("GPU Torch Available = {}".format(torch.cuda.is_available()))
print("Torch Version = {}".format(torch.__version__))

GPU Torch Available = True
Torch Version = 1.8.1+cu101


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
# Loading Dataset
file = '/content/drive/MyDrive/Title Generation/Dataset/Dataset_Title_v2.xlsx' 
df = pd.read_excel(file, names = ['Abstract', 'Domain_Labels', 'Title'])
df = df.drop(['Domain_Labels'], axis=1)
df

In [ ]:
# Use this code to chop off the dataset into 5 or 6 or 7 parts and summarize
parts_df = df[:5000]

In [ ]:
# Summarizer Initialization
summarizer = pipeline(task = "summarization", model = "t5-base", tokenizer= "t5-base", framework = 'pt')

In [ ]:
# Data Summarization (keep max_length = 100 despite of warnings)
train_abstracts = list(parts_df['Abstract'])
summarized_abstracts = []
counter = 0
for text in train_abstracts:
  print('Summarized {} Abstracts from Data'.format(counter+1))
  summarized_text = summarizer(text, min_length = 20, max_length = 100, return_text = True, return_tensors = False)[0]['summary_text']
  summarized_abstracts.append(summarized_text)
  counter += 1

In [ ]:
# Data Preparation into Pandas Dataframe for Model Input
def get_data(dataframe,sum_abs):
  title = list(dataframe['Title'])
  
  raw_data_train = {'Abstract': sum_abs, 'Title': title}
  df = pd.DataFrame(raw_data_train, columns = ['Abstract','Title'])
  return df

data = get_data(parts_df, summarized_abstracts)
data

In [ ]:
data.to_excel("Summarized_Dataset.xlsx")  